# Early Release Process 
## Introduction 
As part of the prison systems overcrowding resolution process, they are required to release some prisoners that have undertaken at least 40% of their sentence.  This section will look at constructing a model where a list of potential release candidates are selected. Release candidates should match those incoming into the prison system - since releasing someone in high 
supervision when you needed a space in low supervison would not be helpful. 

In [ ]:
%pip install -r resources.txt

  Cloning https://github.com/conjure-cp/conjure-notebook.git (to revision v0.0.10) to /tmp/pip-req-build-oa9g5kf2
  Running command git clone --filter=blob:none --quiet https://github.com/conjure-cp/conjure-notebook.git /tmp/pip-req-build-oa9g5kf2
  Running command git checkout -q d58ac9af77e8c8b8d2c3e9633384fb3670fd03e5
  Resolved https://github.com/conjure-cp/conjure-notebook.git to commit d58ac9af77e8c8b8d2c3e9633384fb3670fd03e5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import json
import pandas as pd
import numpy as np

#Now lets try and load in the prisoners to be allocated 
with open('Data/PrisonerList.json', 'r') as f:
    data = json.load(f)

prisoners = data["resident_prisoners"]

with open('Data/PrisonTemplate.json', 'r') as f:
    data = json.load(f)

# Access the prison data
prison = data['prison']

bed_records = []

for section in prison['sections']:
    section_id = section['section_id']
    section_name = section['section_name']
    age_category = section['age_category']
    
    for ward in section['wards']:
        ward_id = ward['ward_id']
        supervision_level = ward['supervision_level']
        
        for wing in ward['wings']:
            wing_id = wing['wing_id']
            sex_assignment = wing['sex_assignment']
            
            for cell in wing['cells']:
                cell_id = cell['cell_id']
                cell_type = cell['cell_type']
                
                for bed in cell['beds']:
                    bed_record = {
                        'section_id': section_id,
                        'section_name': section_name,
                        'age_category': age_category,
                        'ward_id': ward_id,
                        'supervision_level': supervision_level,
                        'wing_id': wing_id,
                        'sex_assignment': sex_assignment,
                        'cell_id': cell_id,
                        'cell_type': cell_type,
                        'bed_id': bed['bed_id'],
                        'occupied': bed['occupied'],
                        'prisoner_id': bed['prisoner_id']
                    }
                    bed_records.append(bed_record)

# Create DataFrame
prison = pd.DataFrame(bed_records)

#In this case we are going to only have those beds that are occupied in the list (similating a scenario where there is no space left in the prison)
prison = prison[prison['occupied'] == True].copy()

#Next lets join the two dataframes togeather by the prisoner id
prisoners_df = pd.DataFrame(prisoners)
merged_df = pd.merge(prisoners_df, prison, left_on='prisoner_id', right_on='prisoner_id', how='inner')
print(merged_df.head())


  prisoner_id              name   sex age_category_x supervision_level_x  \
0      P00001     James MacLeod  Male          Adult                High   
1      P00002     Robert Fraser  Male          Adult                High   
2      P00003  William Campbell  Male          Adult                High   
3      P00004     David Stewart  Male          Adult                High   
4      P00005       John Murray  Male          Adult                High   

  section_id        section_name age_category_y ward_id supervision_level_y  \
0         S1  North Wing Section          Adult    S1W1                High   
1         S1  North Wing Section          Adult    S1W1                High   
2         S1  North Wing Section          Adult    S1W1                High   
3         S1  North Wing Section          Adult    S1W1                High   
4         S1  North Wing Section          Adult    S1W1                High   

  wing_id sex_assignment    cell_id cell_type        bed_id  occupie

In [ ]:
#Drop duiplicate column names 
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]
#Drop the following columns 
columns_to_drop = ['sex_assignment', 'age_category_x', 'supervision_level_x', 'cell_id', 'bed_id', 'occupied']
merged_df = merged_df.drop(columns=columns_to_drop)

#Rename the columns that were kept but had _x or _y added to the end
merged_df = merged_df.rename(columns={
    'age_category_y': 'age_category',
    'supervision_level_y': 'supervision_level'
})
print(merged_df.head())

  prisoner_id              name section_id        section_name age_category  \
0      P00001     James MacLeod         S1  North Wing Section        Adult   
1      P00002     Robert Fraser         S1  North Wing Section        Adult   
2      P00003  William Campbell         S1  North Wing Section        Adult   
3      P00004     David Stewart         S1  North Wing Section        Adult   
4      P00005       John Murray         S1  North Wing Section        Adult   

  ward_id supervision_level wing_id sex_assignment cell_type  
0    S1W1              High   S1W1A           Male    single  
1    S1W1              High   S1W1A           Male    single  
2    S1W1              High   S1W1A           Male    single  
3    S1W1              High   S1W1A           Male    single  
4    S1W1              High   S1W1A           Male    single  


In [ ]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
non_numerical_cols = merged_df.select_dtypes(include=['object', 'bool']).columns
merged_df[non_numerical_cols] = encoder.fit_transform(merged_df[non_numerical_cols])    
print(f"\nDataFrame after encoding non-numerical data:")
merged_df.head()


DataFrame after encoding non-numerical data:


,prisoner_id,name,section_id,section_name,age_category,ward_id,supervision_level,wing_id,sex_assignment,cell_type
0,0.0,118.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
1,1.0,200.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
2,2.0,243.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
3,3.0,44.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
4,4.0,126.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0


In [ ]:
#Now lets try and load in the prisoners to be allocated 
with open('Data/AllocatedPrisonerList.json', 'r') as f:
    data = json.load(f)

prisoners = data["incoming_prisoners"]

# Create a mapping from the original encoder
# Get the column names and their encoded values
col_mapping = {}
for i, col in enumerate(non_numerical_cols):
    categories = encoder.categories_[i]
    col_mapping[col] = {cat: idx for idx, cat in enumerate(categories)}

# Now encode the prisoner data using the mapping
prisoners_encoded = []
index= 1
for prisoner in prisoners:
    prisoner_encoded = prisoner.copy()
    prisoner_encoded['prisoner_id']= int(prisoner['prisoner_id'].lstrip('P'))
    index += 1
    # Map the categorical values using the same encoding
    prisoner_encoded['age_category_encoded'] = col_mapping['age_category'][prisoner['age_category']]
    prisoner_encoded['sex_encoded'] = col_mapping['sex_assignment'][prisoner['sex']]
    prisoner_encoded['supervision_level_encoded'] = col_mapping['supervision_level'][prisoner['supervision_level']]
    # Keep the original categorical columns for later reference
    prisoners_encoded.append(prisoner_encoded)

print(f"\nFirst prisoner after encoding:")
print(prisoners_encoded[0])


First prisoner after encoding:
{'prisoner_id': 256, 'name': 'Angus "Mad Dog" MacDuff', 'sex': 'Male', 'age_category': 'Adult', 'supervision_level': 'High', 'age_category_encoded': 0, 'sex_encoded': 1, 'supervision_level_encoded': 0}


In [ ]:
incoming_prisoners_sex= {}
incoming_prisoners_age = {}
incoming_prisoners_supervision = {}


prisoner_ids = []

for prisoner in prisoners_encoded:
    prisoner_id = int(prisoner['prisoner_id'])
    prisoner_ids.append(prisoner_id)
    incoming_prisoners_sex[prisoner_id] = int(prisoner['sex_encoded'])
    incoming_prisoners_age[prisoner_id] = int(prisoner['age_category_encoded'])
    incoming_prisoners_supervision[prisoner_id] = int(prisoner['supervision_level_encoded']) 


resident_prison_beds_sex= {}
resident_prison_beds_age = {}
resident_prison_beds_supervision = {}
resident_prison_time_served = {}

for index, row in cell_spaces.iterrows():
    # Use sequential bed IDs starting from 1
    resident_prison_beds_sex[bed_counter] = int(row['sex_assignment'])
    resident_prison_beds_age[bed_counter] = int(row['age_category'])                
    resident_prison_beds_supervision[bed_counter] = int(row['supervision_level'])
    resident_prison_time_served[bed_counter] = int(row['time_served'])

#Display the mappings
print(f"\nIncoming Prisoners Sex Mapping: {incoming_prisoners_sex}")
print(f"\nIncoming Prisoners Age Mapping: {incoming_prisoners_age}")
print(f"\nIncoming Prisoners Supervision Mapping: {incoming_prisoners_supervision}")    

print(f"\nResident Prisoner Sex Mapping: {resident_prison_beds_sex}")
print(f"\nResident Prisoner Age Mapping: {resident_prison_beds_age}")
print(f"\nResident Prisoner Supervision Mapping: {resident_prison_beds_supervision}")    
print(f"\nResident Prisoner Time Served Mapping: {resident_prison_time_served}")